In [1]:
!pip install unionllm

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
# install the required packages
from unionllm import unionchat
from datasets import Dataset
from zeval.evaluation import evaluate
from zeval.evaluators import Faithfulness, AnswerRelevancy, ContextRecall, ContextPrecision
# from evalsone import EvalsOne

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# query Coze with a query and get the completion and the context
query = "智商测试"
ideal = ["瑞文标准推理测验（SPM）是一种广受认可的智力测验工具，由英国心理学家瑞文于1938年创制。该测验是一种纯粹的非文字智力测验，广泛应用于无国界的智力/推理能力测试。它基于斯皮尔曼的智力二因素论，认为智力主要由普通因素（G因素）和特殊因素（S因素）构成。瑞文测验主要测量“G”因素，与问题解决、清晰知觉和思维、发现和利用信息以及适应社会生活能力相关。该测验由60张图组成，分为5个单元，考察了知觉辨别、类同比较、比较推理、系列关系推理、抽象推理运算等方面的思维能力。"]
messages = [
    {"role": "user", "content": "智商测试"}
]

params = {
    "provider": "coze",
    "model": "coze",
    "bot_id": "7355829428481769477",
    "api_key": "pat_hodyfcRog0CfCAfEMtKAueI1I8yD2tDuDBSrvSRoXhKm1xLCw0HDiePtEVgRFHzt",
    "stream": False,
    "details": True
}

response = unionchat(messages=messages, **params)
print(response)

ModelResponse(id='3f6d5e8142eb4d5984a88b93b4e007d0', choices=[Choices(finish_reason='stop', index=0, message=Message(content='瑞文标准智力测验是一款广受认可的智力测验工具，它是一种纯粹的非文字智力测验。在这里，你可以进行智力测验：\n\n[点击这里进行瑞文标准智力测验](http://p.melbacastillo.top/pquiz/rend/2725039?retake=1\\u0026cstr=5c987b7c4001f)\n\n请记住，瑞文智力测验结果与测试参与者在测验时的精神状态，周边环境等多种因素有关，波动范围大约在3至15分左右。', role='assistant'))], created=1716650197, model='', object='chat.completion', system_fingerprint=None, usage=Usage(), context=[])


In [8]:
# create a dataset from the response for evaluation
sampled = response.choices[0].message.content
context = []
for context_item in response.context:
    context.append(context_item.content)

dataset = Dataset.from_dict({
    'question': [query],
    'answer': [sampled], 
    'context': [context],
    'ideal': [ideal]
})
print(dataset['question'])

Dataset({
    features: ['question', 'answer', 'context', 'ideal'],
    num_rows: 1
})


In [5]:
# specify the evaluation parameters
eval_params = {
    "provider": "zhipuai",
    "model": "glm-4",
    "api_key": "38dc046c622b6ec1b9bfa0413e6ca2ee.2Yn3uFr8j74pMOvK",
    "temperature": 0.1,
    "stream": False
}

In [6]:
# evaluate the dataset with the specified evaluator and print the results
faithfulness_results = evaluate(dataset, [Faithfulness], eval_params)
print(faithfulness_results)
print(faithfulness_results.reasoning)

Evaluating with type


KeyError: 'question'

In [ ]:
eo_client = EvalsOne(api_key='your_api_key_here')
kwargs = {'messages': 'example message', 'completion': 'example completion'}
response = eo_client.add_sample(**kwargs)
print(response)